<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/novel_dialogue_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat凉宫春日 Chat-Haruhi-Suzumiya

[![Code License](https://img.shields.io/badge/Code%20License-Apache_2.0-green.svg)]()
[![Data License](https://img.shields.io/badge/Data%20License-CC%20By%20NC%204.0-red.svg)]()
[![Huggingface Gradio](https://img.shields.io/static/v1?label=Demo&message=Huggingface%20Gradio&color=orange)](https://huggingface.co/spaces/silk-road/ChatHaruhi)


本脚本是李鲁鲁开发的自动json剧本分析工具

用于分析

https://github.com/LC1332/Prophet-Andrew-Ng/blob/main/langchain/%E6%9D%8E%E9%B2%81%E9%B2%81%E5%AD%A6LangChain_25_Kor%E4%BF%A1%E6%81%AF%E6%8A%BD%E5%8F%96.ipynb

这个脚本抓取后的输出

---

**Chat凉宫春日**是模仿凉宫春日等一系列动漫人物，使用近似语气、个性和剧情聊天的语言模型，

<details>
  <summary> 本项目由李鲁鲁，冷子昂，闫晨曦，封小洋，scixing，沈骏一，Aria Fei, 米唯实, 吴平宇, 贾曜恺等开发。 </summary>

李鲁鲁发起了项目，并完成了最早的版本，在多个微信群实现了测试。完成了训练数据自动生成对话部分，设计了整体的路线，并撰写报告。

冷子昂负责了每一个阶段的Gradio开发，以及每个部分的功能整合和架构设计。

闫晨曦一开始将李鲁鲁的notebook重构为app.py，参与了WebUI的embedding部分重构整合。

封小洋进行了中文转日文模型的选型，完成了针对台词抽取图片的工具。整合了声纹识别。即将继续参加台本工具的开发。

scixing实践了VITS语音，完成了台词对应的语音抽取，实现了第一个版本的声纹分类。

沈骏一实现了使用ChatGLM2 finetune实验

Aria(Yaying Fei)对接了whisper到台本工具。即将继续参加台本工具的开发。

米唯实实现了Chat哆啦A梦的分支版本

吴平宇部署了ChatGLM2的训练程序，并提供了训练的计算资源。

贾曜恺开发了一个Vue前端实现方案

</details>

计划特征

- [x] 读取整个文件夹中的jsonl进行组织
- [x] 看看dialogue - action的模态是否符合预期
- [x] 看看每个高频角色的出现时间
- [x] 尝试抽取每个角色和韦小宝的互动

先载入文件

In [1]:
# # mount google drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [116]:
# !cp /content/drive/MyDrive/GPTData/weixiaobao_extract.zip "/content"
!wget https://github.com/LC1332/Prophet-Andrew-Ng/raw/main/langchain/weixiaobao_extract.zip /content

--2023-07-29 11:26:25--  https://github.com/LC1332/Prophet-Andrew-Ng/raw/main/langchain/weixiaobao_extract.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/weixiaobao_extract.zip [following]
--2023-07-29 11:26:25--  https://raw.githubusercontent.com/LC1332/Prophet-Andrew-Ng/main/langchain/weixiaobao_extract.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1618599 (1.5M) [application/zip]
Saving to: ‘weixiaobao_extract.zip.1’

weixiaobao_extract. 100%[===================>]   1.54M  --.-KB/s    in 0.06s   

2023-07-29 11:26:26 (26.6 MB/s) - ‘weixiaobao

In [8]:
from zipfile import ZipFile


# 解压缩
with ZipFile('/content/weixiaobao_extract.zip'.split('/')[-1], 'r') as zip:
  zip.extractall('/content')

print('Zip file extracted to /content')

Zip file extracted to /content


In [9]:
import os

save_folder = "/content/weixiaobao_extract"

if not os.path.exists(save_folder):
    print('unfound folder', save_folder)


import os
import json
from tqdm import tqdm

max_n = 1876

data = []

for i in tqdm(range(0,max_n)):
    save_name = os.path.join(save_folder, f"{i}.txt")

    if not os.path.exists(save_name):
        print(f"{save_name} not found")
        continue

    ## read save_name

补全这段代码， save_folder / {i}.txt中，存储了jsonl格式的数据，形如

```
{"role": "韦小宝", "dialogue": "这些客人都是汉人罢", "action": "对话"}
{"role": "韦春芳", "dialogue": "汉人自然有，满洲官儿也有，还有蒙古的武官呢", "action": "对话"}
{"role": "韦小宝", "dialogue": "外国鬼子没有罢", "action": "对话"}
{"role": "韦春芳", "dialogue": "你当你娘是烂婊子吗连外国鬼子也接辣块妈妈，罗刹鬼、红毛鬼到丽春院来，老娘用大扫帚拍了出去", "action": "对话"}
{"role": "韦小宝", "dialogue": "那很好", "action": "对话"}
{"role": "韦春芳", "dialogue": "怎么没有那个西藏喇嘛，上床之前一定要念经，一面念经，眼珠子就骨溜溜的瞧着我。你一双眼睛贼忒嘻嘻的，真像那个喇嘛", "action": "对话"}
```

补全这段代码，将文件中的内容读取后，以list of json的格式保存到data中

In [14]:

import os
import json
from tqdm import tqdm

max_n = 1876

datas = []

for i in tqdm(range(0,max_n)):
    save_name = os.path.join(save_folder, f"{i}.txt")

    if not os.path.exists(save_name):
        print(f"{save_name} not found")
        continue

    ## read save_name
    with open(save_name, 'r') as f:
        for line in f:
            datas.append(json.loads(line))

100%|██████████| 1876/1876 [00:00<00:00, 11011.51it/s]

/content/weixiaobao_extract/0.txt not found
/content/weixiaobao_extract/1.txt not found


In [15]:
print(len(datas))

17626


datas是一个python的list of json

里面的元素格式如下
{'role': '吕留良', 'dialogue': '晚村兄，有一件要紧事，特来和你商议。', 'action': '对话'}

实现一段python程序，分析datas中有多少种不重复的role，

建立一个role_names集合，统计不同的role的名字

以及统计datas中有多少种长度小于5的action。输出所有的不同的role名字的列表

In [36]:
import json

role_names = set()
short_actions = set()
action_for_dialogue = []

for i in range(0,len(datas)):
  data = datas[i]
  role = data['role']
  action = data['action']
  dialogue = data['dialogue']

  if role not in role_names:
    # print(f"found new role {role} on line {i}")
    role_names.add(role)

  if len(action) < 5:
    if action not in short_actions:
      short_actions.add(action)

      if dialogue != "":
         print(f"found new action {action} on line {i}")
         print("对话不为空",dialogue)
         action_for_dialogue.append(action)

# print(f"共有{len(role_names)}种角色:")
# print(list(role_names))

# print(f"共有{short_actions}种长度小于5的action")

print(action_for_dialogue)

found new action 独白 on line 0
对话不为空 咱们做老百姓的，总是死路一条。未知鹿死谁手，只不过未知是谁来杀了这头鹿，这头鹿，却是死定了的。
found new action 对话 on line 3
对话不为空 晚村兄，有一件要紧事，特来和你商议。
found new action  on line 228
对话不为空 独白
found new action 对句 on line 229
对话不为空 十年一觉扬州梦，赢得青楼薄幸名
found new action 倒飞出来，摔在地下 on line 242
对话不为空 哎唷
found new action 被摔出来 on line 245
对话不为空 连声呼叫
found new action 骂道 on line 251
对话不为空 他妈的臭婊子，有什么好笑
found new action 惨呼 on line 259
对话不为空 啊
found new action 惊叫 on line 260
对话不为空 啊
found new action 大叫 on line 554
对话不为空 辣块妈妈
found new action 对骂 on line 1929
对话不为空 老婊子、烂婊子、臭婊子
found new action 急冲出房，其中二人跃上了屋顶 on line 2449
对话不为空 是
found new action 大为惊讶 on line 4392
对话不为空 啊
found new action 坐在禅房中 on line 6158
对话不为空 （无对话）
found new action 叫道 on line 9086
对话不为空 啊哟，不好，经书上有毒
found new action - on line 9232
对话不为空 -
found new action 叫声 on line 9645
对话不为空 啊啊
found new action 翻身入水 on line 11673
对话不为空 啊
found new action 回答洪夫人 on line 12289
对话不为空 是
found new action 插口 on line 15400
对话不为空 你自己的老公，还不也死了
found new action 怒骂 on line 154

datas是一个python的list of json

里面的元素格式如下
{'role': '吕留良', 'dialogue': '晚村兄，有一件要紧事，特来和你商议。', 'action': '对话'}

实现一段python程序，分析datas中有多少种不重复的role，

分析每个角色出现的次数

In [119]:
from collections import defaultdict
from heapq import nlargest

role_count = defaultdict(int)

for data in datas:
  role = data['role']
  role_count[role] += 1



n = 100

# 输出出现次数最多的50个角色
print("出现次数最多的",n,"个角色:")

dspid = 0
for role, count in nlargest(n, role_count.items(), key=lambda x: x[1]):
  dspid = dspid + 1
  if dspid % 3 == 0:
    print(f"{role}: {count} 次",end = "\n")
  else:
    print(f"{role}: {count} 次",end="\t")
print()

top_roles = nlargest(n, role_count.items(), key=lambda x: x[1])

org_top_roles = top_roles
remove_roles = ['众人', '那人', '老者']
top_roles = [x[0] for x in top_roles if (x[0] not in remove_roles)]

print(top_roles)


出现次数最多的 100 个角色:
韦小宝: 7129 次	康熙: 755 次	双儿: 315 次
海老公: 296 次	太后: 293 次	公主: 287 次
阿珂: 276 次	陈近南: 240 次	茅十八: 239 次
郑克塽: 203 次	白衣尼: 194 次	方怡: 174 次
澄观: 151 次	洪教主: 127 次	洪夫人: 126 次
施琅: 121 次	沐剑屏: 114 次	小郡主: 110 次
胖头陀: 106 次	徐天川: 104 次	索额图: 101 次
多隆: 101 次	吴之荣: 94 次	吴立身: 89 次
九难: 89 次	费要多罗: 89 次	众人: 85 次
陶红英: 77 次	苏菲亚: 75 次	吴三桂: 74 次
陈圆圆: 73 次	康亲王: 72 次	白寒枫: 70 次
刘一舟: 70 次	吴六奇: 69 次	吴应熊: 65 次
张康年: 62 次	陆高轩: 62 次	桑结: 59 次
瘦头陀: 59 次	苏荃: 57 次	归二娘: 56 次
杨溢之: 53 次	冯锡范: 53 次	澄光: 51 次
那人: 50 次	老者: 49 次	钱老本: 49 次
陆先生: 49 次	风际中: 48 次	李自成: 47 次
柳大洪: 46 次	陶宫娥: 46 次	夏国相: 44 次
王进宝: 43 次	韦春芳: 42 次	鳌拜: 40 次
行痴: 40 次	玄贞道人: 39 次	苏冈: 39 次
病汉: 39 次	李西华: 38 次	吕留良: 37 次
-: 37 次	喇嘛: 37 次	建宁公主: 37 次
林兴珠: 37 次	顾炎武: 36 次	查伊璜: 36 次
温有道: 35 次	樊纲: 34 次	玄贞: 33 次
洪安通: 33 次	高彦超: 31 次	归钟: 31 次
侍卫: 30 次	沐剑声: 30 次	亲兵: 28 次
太监: 28 次	女子: 28 次	赵齐贤: 28 次
那女郎: 28 次	慕天颜: 28 次	图尔布青: 28 次
李力世: 27 次	葛尔丹: 27 次	赵良栋: 27 次
归辛树: 27 次	军官: 26 次	温有方: 26 次
钱老板: 26 次	: 26 次	矮胖子: 26 次
小玄子: 25 次	宫女: 25 次	关安基: 25 次
马彦超: 25 次	巴颜: 25 次	马超兴: 24 次
柳燕

In [113]:
replace_map = {}

replace_map['公主'] = '建宁公主'
replace_map['小玄子'] = '康熙'
replace_map['苏荃'] = '洪夫人'
replace_map['九难'] = '白衣尼'
replace_map['沐剑屏'] = '小郡主'

In [114]:
disp_cut_n = 34

org_top_roles = top_roles

top_roles = org_top_roles[:disp_cut_n]

offset = 0

for i in range(0,len(top_roles)):
    if top_roles[i] in replace_map.keys():
        print('found ', top_roles[i])
        if replace_map[top_roles[i]] in top_roles:
            while org_top_roles[disp_cut_n + offset] in top_roles:
                offset += 1
            top_roles[i] = org_top_roles[disp_cut_n + offset]
            print('replace with ',top_roles[i] )

        else:
            top_roles[i] = replace_map[top_roles[i]]
            print('replace ', top_roles[i])


print(top_roles)

roles = []

dialogue_in_time = []

for data in datas:

    role = data['role']

    if role in replace_map.keys():
        role = replace_map[role]

    if role in top_roles:
        roles.append(role)
    else:
        roles.append('其他')

    if data['dialogue'] == '':
        dialogue_in_time.append(data['action'])
    else:
        dialogue_in_time.append( data['dialogue'] )

role_name = top_roles
role_name.append('其他')
role_in_time = roles

found  公主
replace  建宁公主
found  沐剑屏
replace with  吴应熊
found  九难
replace with  张康年
['韦小宝', '康熙', '双儿', '海老公', '太后', '建宁公主', '阿珂', '陈近南', '茅十八', '郑克塽', '白衣尼', '方怡', '澄观', '洪教主', '洪夫人', '施琅', '吴应熊', '小郡主', '胖头陀', '徐天川', '索额图', '多隆', '吴之荣', '吴立身', '张康年', '费要多罗', '陶红英', '苏菲亚', '吴三桂', '陈圆圆', '康亲王', '白寒枫', '刘一舟', '吴六奇']


我希望使用python的plotly绘制一张图

role_name是一个长度为的list of string，记录了20个角色的名字，如'韦小宝', '康熙', '双儿', '海老公', ....

role_in_time是一个长度为10000的list of string，里面的元素都是role_name中的某个字符串

我希望用python的plotly绘制一张散点图

纵轴为高度20，纵轴左侧需要显示每个角色的位置

横轴为时间，宽度为10000

如果role_in_time的第i个角色，在role_name中的序号为j，则在i,j处标记一个散点

我希望每个角色用不同的颜色进行表示

In [115]:
import plotly.express as px

# 创建映射字典,角色名字到编号
name_to_num = {name:i for i,name in enumerate(role_name)}

# 将角色名字序列转换为编号序列
role_index = [name_to_num[name] for name in role_in_time]

# 生成不同颜色的颜色序列
colors = px.colors.qualitative.Dark24
while len(colors) < len(role_name):
    colors = colors + colors


# 绘制散点图
fig = px.scatter(
    x=list(range(len(role_in_time))),
    y=role_index,
    color=[colors[i] for i in role_index],
    hover_data={'t': dialogue_in_time}
)

# 设置y轴为角色名字
fig.update_yaxes(ticktext=role_name, tickvals=list(range(len(role_name))))


# 设置坐标轴范围
fig.update_xaxes(range=[0, len(role_in_time)])
# fig.update_yaxes(range=[0, len(role_name)])

# 设置坐标轴标题
fig.update_layout(
    xaxis_title='时间',
    yaxis_title='角色'
)

fig.show()

In [117]:
print(len(datas))

17626


In [120]:
!pip install -q tiktoken

import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.7 MB/s eta 0:00:00


In [154]:
def count_token( data ):
    role_token = len(enc.encode( data['role'] ))
    if data['dialogue'] != '':
        return role_token + len(enc.encode( data['dialogue'] )) + 5
    else:
        return role_token + len(enc.encode( data['action'] )) + 5

print( count_token(datas[120]))

25


接下来我们希望对主角的话进行抽取

In [161]:
from re import I
from ast import Continue

top_roles = [x[0] for x in org_top_roles if x[0] != '韦小宝']
print(top_roles)

current_id = 0

role_name = "韦小宝"

action_for_dialogue = ['对话', '对句', '骂道', '惨呼', '惊叫', '大叫', '对骂', '叫道',  '叫声', '插口', '怒骂','对方怡说']
action_for_monologue = ['独白']

status = "FIND_NEXT_CHUNK"

current_chunk = []
result_chunks = []

ALLOW_PREFIX_MAX_LINES = 3

tmp_count = 0
ALLOW_PREFIX_NOISE = 1

MAX_SURFIX_LINE_SEARCH = 5
ALLOW_SURFIX_NOISE = 1

MAX_DIALOGUE_TOKEN = 400
MAX_DIALOGUE_TOKEN_PREVENT = 1000

MAX_ROLE_ALLOW = 5


while current_id<len(datas):

    if current_id % 1000 == 0:
        print(current_id)

    if status == "FIND_NEXT_CHUNK":
        # 先找到current_id下一个中，主角进行对话的句子
        data = datas[current_id]
        if data['role'] == role_name and data['action'] in action_for_dialogue:
            status = "START_ONE_CHUNK"
            # print("start chunk, dialogue = ", data['dialogue'])
        else:
            current_id += 1
            continue


    if status == "START_ONE_CHUNK":

        # print("start one chunk")

        chunk_start_id = -1

        # 从current_id向前，寻找第一个非主角，但是在top_roles中的角色，作为起始
        for i in range(current_id-1,current_id - ALLOW_PREFIX_MAX_LINES , -1):
            if datas[i]['role'] in top_roles:
                chunk_start_id = i
                break

        if chunk_start_id == -1:
            # print('没有为台词', data['dialogue'] ,'找到合适的前继句子')
            current_id += 1
            status = "FIND_NEXT_CHUNK"
            continue

        prefix_noise_count = 0
        for i in range(chunk_start_id + 1, current_id):
            if datas[i]['role'] != role_name:
                prefix_noise_count += 1

        if prefix_noise_count > ALLOW_PREFIX_NOISE:
            # print('没有为台词', data['dialogue'] ,'找到合适的前继句子')
            current_id += 1
            status = "FIND_NEXT_CHUNK"
            continue

        current_chunk = []
        for i in range(chunk_start_id, current_id+1):
            current_chunk.append(datas[i])

        current_role_pool = []
        for data in current_chunk:
            current_role_pool.append(data['role'])

        current_count = 0

        for data in current_chunk:
            current_count += count_token(data)

        if current_count > MAX_DIALOGUE_TOKEN_PREVENT:
            # print('没有为台词', data['dialogue'] ,'找到合适的前继�
            current_id += 1
            status = "FIND_NEXT_CHUNK"
            continue

        status = "GRAB_DIALOGUE"

    if status == "GRAB_DIALOGUE":
        current_id += 1

        end_id = -1

        #考虑之后的台词中，是否出现主角
        for id in range(current_id, current_id + MAX_SURFIX_LINE_SEARCH):
            if datas[id]['role'] == role_name:
                end_id = id
                break

        if end_id == -1:
            # 进入结算环节
            status = "END_NEW_CHUNK"
            continue

        #统计noise
        surfix_noise_count = 0
        for id in range(current_id, end_id):
            if datas[id]['role'] not in top_roles:
                surfix_noise_count += 1

        if surfix_noise_count > ALLOW_SURFIX_NOISE:
            # 进入结算环节
            status = "END_NEW_CHUNK"
            continue

        #统计增加了多少角色
        for id in range(current_id, end_id):
            if datas[id]['role'] not in current_role_pool:
                current_role_pool.append(datas[id]['role'])

        if len(current_role_pool) > MAX_ROLE_ALLOW:
            # 进入结算环节
            status = "END_NEW_CHUNK"
            continue

        #统计增加了多少token
        for id in range(current_id, end_id+1):
            current_count += count_token(datas[id])



        if current_count > MAX_DIALOGUE_TOKEN:
            # 进入结算环节
            status = "END_NEW_CHUNK"
            continue

        #不然 则考虑将对话加入到chunk中
        for id in range(current_id, end_id+1):
            current_chunk.append(datas[id])

        current_count = 0

        for data in current_chunk:
            current_count += count_token(data)

        current_id = end_id + 1

    if status == "END_NEW_CHUNK":
        result_chunks.append(current_chunk)
        current_chunk = []
        status = "FIND_NEXT_CHUNK"






    current_id += 1
    # if current_id > 1000:
    #     break

print('tmp_count = ',tmp_count)

['康熙', '双儿', '海老公', '太后', '公主', '阿珂', '陈近南', '茅十八', '郑克塽', '白衣尼', '方怡', '澄观', '洪教主', '洪夫人', '施琅', '沐剑屏', '小郡主', '胖头陀', '徐天川', '索额图', '多隆', '吴之荣', '吴立身', '九难', '费要多罗', '众人', '陶红英', '苏菲亚', '吴三桂', '陈圆圆', '康亲王', '白寒枫', '刘一舟', '吴六奇', '吴应熊', '张康年', '陆高轩', '桑结', '瘦头陀', '苏荃', '归二娘', '杨溢之', '冯锡范', '澄光', '那人', '老者', '钱老本', '陆先生', '风际中', '李自成', '柳大洪', '陶宫娥', '夏国相', '王进宝', '韦春芳', '鳌拜', '行痴', '玄贞道人', '苏冈', '病汉', '李西华', '吕留良', '-', '喇嘛', '建宁公主', '林兴珠', '顾炎武', '查伊璜', '温有道', '樊纲', '玄贞', '洪安通', '高彦超', '归钟', '侍卫', '沐剑声', '亲兵', '太监', '女子', '赵齐贤', '那女郎', '慕天颜', '图尔布青', '李力世', '葛尔丹', '赵良栋', '归辛树', '军官', '温有方', '钱老板', '', '矮胖子', '小玄子', '宫女', '关安基', '马彦超', '巴颜', '马超兴', '柳燕']
0
2000
3000
6000
9000
10000
12000
13000
16000
tmp_count =  0


In [162]:
print(len(result_chunks))


1048


In [177]:
action_for_dialogue = ['对话', '对句', '骂道', '惨呼', '惊叫', '大叫', '对骂', '叫道',  '叫声', '插口', '怒骂','对方怡说']
action_for_monologue = ['独白']

def data2str( data ):
    role = data['role']
    dialogue = data['dialogue']
    action = data['action']

    if action in action_for_dialogue:
        return role + ':「' + dialogue + '」'
    elif action in action_for_monologue:
        return '旁白' + ':' + role + '想"' + dialogue + '"'
    else:
        return ':' + role + action

def chunk2str( chunk ):
    result = ''
    strs = []
    for data in chunk:
        strs.append(data2str(data))

    flag = len(chunk)-1

    for i in range(flag, 0 , -1):
        if len(strs[i]) < len(role_name):
            continue
        if strs[i][:len(role_name)] == role_name:
            flag = i
            break

    for i in range(0, flag+1):
        result = result + strs[i] + '\n'
    return result




for i in range(0,len(chunk2str)):
    my_str = chunk2str(result_chunks[i])

这段程序会打印很多不同的字符串

请为我改写这段程序，在/content 建立一个名叫texts的文件夹，

将每个iter的my_str保存到一个单独的.txt文件中，注意字符串中的中文

然后将整个文件夹压缩为一个weixiaobao_text.zip的压缩包保存在/content

In [185]:
import os
import zipfile

os.makedirs('/content/texts', exist_ok=True)

for i in range(0,len(result_chunks)):
  my_str = chunk2str(result_chunks[i])

  with open(f'/content/texts/text_{i}.txt', 'w', encoding='utf-8') as f:
    f.write(my_str)

zip_path = '/content/weixiaobao_text.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
  for filename in os.listdir('/content/texts'):
    zipf.write('/content/texts/' + filename)

print('Zipped folder saved to', zip_path)

Zipped folder saved to /content/weixiaobao_text.zip
